In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install -U transformers accelerate datasets scikit-learn pandas matplotlib notebook

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.metrics import accuracy_score, f1_score, classification_report

c:\.code\python_service\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 4050 Laptop GPU


In [3]:
# === 1. Load dan Persiapan Dataset ===

# Load data
path_train = './dataset/cleaned_train.csv'
path_test = './dataset/cleaned_test.csv'

train_df = pd.read_csv(path_train).sample(n=24000, random_state=42)
test_df = pd.read_csv(path_test).sample(n=6000, random_state=42)

# Map ke label numerik
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
inv_label_map = {v: k for k, v in label_map.items()}

train_df['label'] = train_df['review_class'].map(label_map)
test_df['label'] = test_df['review_class'].map(label_map)

# Print contoh
print(train_df[['clean_text', 'review_class', 'label']].head())
print(test_df[['clean_text', 'review_class', 'label']].head())


                                              clean_text review_class  label
3111   pertama kali make up lipstick pertama aku warn...     positive      2
18679  yes this is my all time favorite hair oil boto...     positive      2
17472  sebenernya aku cuma pernah beberapa kali nyoba...     negative      0
21451  tarik sama produk liat tutorial stephanie rose...     positive      2
20800  aku bilang msh mula bgt dlm dunia permake upan...     positive      2
                                             clean_text review_class  label
1782  inget banget dulu pas smp dove pernah punya fa...     negative      0
3917  aplikasiinnya enak banget busa dgn desain kaya...     positive      2
221   formula ringan cukup bagus buat nahan minyak g...     positive      2
2135  semua varian pixy stick deodorant entah cuma y...     positive      2
5224  sebenernya cinta banget sama semua varian tone...     positive      2


In [4]:
# === 2. Konversi ke Dataset HuggingFace ===

train_dataset = Dataset.from_pandas(train_df[['clean_text', 'label']].rename(columns={"clean_text": "text"}))
test_dataset = Dataset.from_pandas(test_df[['clean_text', 'label']].rename(columns={"clean_text": "text"}))

In [5]:
# === 3. Tokenisasi ===

checkpoint = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, max_length=512)

train_tokenized = train_dataset.map(tokenize_fn, batched=True)
test_tokenized = test_dataset.map(tokenize_fn, batched=True)

Map: 100%|██████████| 6000/6000 [00:00<00:00, 24360.21 examples/s]


In [6]:
# === 4. Load Model dan Setup Trainer ===

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="none",
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# === 5. Fungsi Evaluasi ===

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1_macro": f1_score(labels, predictions, average="macro"),
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\user\AppData\Local\Temp\ipykernel_34720\3029752033.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
# === 6. Training dan Evaluasi ===

trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.562300,0.538053,0.776500,0.608709
2,0.471800,0.550259,0.769000,0.619363
3,0.367200,0.645167,0.776167,0.633973
4,0.245800,0.765956,0.767667,0.623110
5,0.158600,1.020073,0.760167,0.620688


{'eval_loss': 0.5380529165267944,
 'eval_accuracy': 0.7765,
 'eval_f1_macro': 0.6087092572926717,
 'eval_runtime': 154.6199,
 'eval_samples_per_second': 38.805,
 'eval_steps_per_second': 2.425,
 'epoch': 5.0}

In [9]:
# === 7. Evaluasi Lengkap dengan Label String ===

predictions = trainer.predict(test_tokenized)
predicted_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids

predicted_classes = [inv_label_map[i] for i in predicted_labels]
true_classes = [inv_label_map[i] for i in true_labels]

accuracy = accuracy_score(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print("Accuracy:", accuracy)
print("F1-score:", f1)
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=["negative", "neutral", "positive"]))


Accuracy: 0.7765
F1-score: 0.7568965713191518
Classification Report:
              precision    recall  f1-score   support

    negative       0.64      0.52      0.57       697
     neutral       0.46      0.31      0.37      1092
    positive       0.84      0.94      0.89      4211

    accuracy                           0.78      6000
   macro avg       0.65      0.59      0.61      6000
weighted avg       0.75      0.78      0.76      6000



In [10]:
# === 8. Simpan Model ===

save_path = "./model/indobert_sentiment_model"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

('./model/indobert_sentiment_model\\tokenizer_config.json',
 './model/indobert_sentiment_model\\special_tokens_map.json',
 './model/indobert_sentiment_model\\vocab.txt',
 './model/indobert_sentiment_model\\added_tokens.json',
 './model/indobert_sentiment_model\\tokenizer.json')

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

load_path = ".\model\indobert_sentiment_model"

model = AutoModelForSequenceClassification.from_pretrained(load_path)
tokenizer = AutoTokenizer.from_pretrained(load_path)

In [16]:
from transformers import pipeline

predictor = pipeline("text-classification", model=model, tokenizer=tokenizer)

texts = [
    "produk jelek dan bodoh",
    "Saya sangat senang dengan produk ini!",
    "Aku suka produk desainnya, tapi ini jelek kualitasnya",
    "Produk ini luar biasa, saya akan membelinya lagi!",
    "Pengiriman sangat cepat dan sesuai harapan.",
    "Pelayanan pelanggan sangat membantu dan ramah.",
    "Saya puas dengan kualitas dan harganya.",
    "Desainnya elegan dan sangat nyaman dipakai.",

    "Pelayanannya mengecewakan",
    "banyak fitur bermasalah dan menghambat",
    "Barang datang rusak dan tidak sesuai deskripsi.",
    "Sangat kecewa, tidak akan beli lagi di sini.",
    "Aplikasi sering crash dan membuat frustasi.",
    "Kualitasnya buruk, terasa murahan.",
    "Pengalaman belanja yang sangat buruk.",

    "Produk sesuai deskripsi.",
    "Masih perlu dicoba beberapa hari ke depan.",
    "Barang diterima. Belum diuji.",
    "Warnanya beda sedikit dari foto.",
    "Tidak ada masalah berarti sejauh ini."
]

results = predictor(texts)

for text, result in zip(texts, results):
    print(f"Teks: {text}")
    print(f"Label: {result['label']}")
    print(f"Confidence: {result['score']:.2f}\n")

Device set to use cuda:0


Teks: produk jelek dan bodoh
Label: negative
Confidence: 0.70

Teks: Saya sangat senang dengan produk ini!
Label: positive
Confidence: 0.86

Teks: Aku suka produk desainnya, tapi ini jelek kualitasnya
Label: neutral
Confidence: 0.44

Teks: Produk ini luar biasa, saya akan membelinya lagi!
Label: positive
Confidence: 0.66

Teks: Pengiriman sangat cepat dan sesuai harapan.
Label: positive
Confidence: 0.82

Teks: Pelayanan pelanggan sangat membantu dan ramah.
Label: positive
Confidence: 0.89

Teks: Saya puas dengan kualitas dan harganya.
Label: positive
Confidence: 0.71

Teks: Desainnya elegan dan sangat nyaman dipakai.
Label: positive
Confidence: 0.81

Teks: Pelayanannya mengecewakan
Label: negative
Confidence: 0.69

Teks: banyak fitur bermasalah dan menghambat
Label: negative
Confidence: 0.44

Teks: Barang datang rusak dan tidak sesuai deskripsi.
Label: negative
Confidence: 0.73

Teks: Sangat kecewa, tidak akan beli lagi di sini.
Label: negative
Confidence: 0.74

Teks: Aplikasi sering c